In [1]:
import pandas as pd
f = open('建模数据.csv')
file = pd.read_csv(f, encoding = 'utf-8')
file.head()

,Direction,Elevator,Floor,Layout,Region,Renovation,Size,age,Price
0,3,2,6,16,0,3,75.0,33,780.0
1,9,0,6,9,0,3,60.0,33,705.0
2,10,1,16,16,0,0,210.0,25,1400.0
3,7,2,7,2,0,3,39.0,17,420.0
4,7,1,19,10,0,3,90.0,11,998.0


In [2]:
col_list = ['Direction', 'Elevator', 'Floor', 'Layout', 'Region', 'Renovation', 'Size', 'age', 'Price']
file_a = file[col_list]
file_a.head()

,Direction,Elevator,Floor,Layout,Region,Renovation,Size,age,Price
0,3,2,6,16,0,3,75.0,33,780.0
1,9,0,6,9,0,3,60.0,33,705.0
2,10,1,16,16,0,0,210.0,25,1400.0
3,7,2,7,2,0,3,39.0,17,420.0
4,7,1,19,10,0,3,90.0,11,998.0


使用toad库进行逐步回归筛选变量，以7:3比例划分训练集和测试集

In [3]:
import toad

data_train = file_a.iloc[0: int(len(file_a) * 0.8)]

In [4]:
stepwise_selected = toad.selection.stepwise(data_train, target = 'Price', estimator = 'ols', direction = 'both', criterion = 'aic', exclude = [])

In [5]:
feature_list = [i for i in stepwise_selected.columns if i not in ['Price']]
feature_list

['Direction', 'Elevator', 'Floor', 'Layout', 'Renovation', 'Size', 'age']

上面的结果是入模变量，导入statsmodels库，进行OLS建模

In [6]:
import statsmodels.api as sm

In [7]:
feature_list.append('Price')
data_train = data_train[feature_list]
data_train.head()

,Direction,Elevator,Floor,Layout,Renovation,Size,age,Price
0,3,2,6,16,3,75.0,33,780.0
1,9,0,6,9,3,60.0,33,705.0
2,10,1,16,16,0,210.0,25,1400.0
3,7,2,7,2,3,39.0,17,420.0
4,7,1,19,10,3,90.0,11,998.0


In [9]:
x = sm.add_constant(data_train.iloc[:,:len(feature_list) - 1]) #生成自变量
y = data_train['Price'] #生成因变量
model = sm.OLS(y, x) #生成模型

In [10]:
result = model.fit()

In [11]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     3086.
Date:                Fri, 30 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:51:57   Log-Likelihood:            -1.2208e+05
No. Observations:               17720   AIC:                         2.442e+05
Df Residuals:                   17712   BIC:                         2.442e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -272.3428     10.776    -25.273      0.000    -293.465    -251.221
Direction     -1.0507      0.579     -1.816      0.069      -2.185       0.084
Elevator       8.9828      2.407      3.733      0.000       4.266      13.700
Floor          7.9654      0.266     29.968      0.000       7.444       8.486
Layout        -2.1660      0.398     -5.444      0.000      -2.946      -1.386
Renovation    20.7224      1.796     11.537      0.000      17.202      24.243
Size           5.6892      0.056    102.044      0.000       5.580       5.799
age            6.7852      0.249     27.261      0.000       6.297       7.273
==============================================================================
Omnibus:                     8448.261   Durbin-Watson:                   1.252
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           192525.856
Skew:                           1.778   Prob(JB):                         0.00
Kurtosis:                      18.751   Cond. No.                         694.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""